In [6]:
import pandas as pd
import numpy as np
import ast


In [7]:
movies_df = pd.read_csv('tmdb_5000_movies.csv')
credits_df = pd.read_csv('tmdb_5000_credits.csv')

In [8]:
movies_df = movies_df.merge(credits_df, on='title')

In [9]:
# genres
# movie_id
# title
# cast
# crew
# overview
# keywords

In [10]:
movies_df = movies_df[['movie_id','title','overview','genres','keywords','cast','crew']]

In [11]:
movies_df.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [12]:
movies_df.dropna(inplace=True)

In [13]:
movies_df.duplicated().sum()

0

In [33]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [34]:
new_df['tags'][0]


'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d action adventure fantasy sciencefiction samworthington zoesaldana sigourneyweaver jamescameron'

In [16]:
def converter_genres(value):
    l = []
    for ele in ast.literal_eval(value):
        l.append(ele['name'])
    return l    
    

In [17]:
movies_df['genres'] = movies_df['genres'].apply(converter_genres)

In [18]:
movies_df['keywords'] = movies_df['keywords'].apply(converter_genres)

In [19]:
def converter_cast(value):
    l = []
    for ele in ast.literal_eval(value):
        if ele['order'] in [0,1,2]:
            l.append(ele['name'])
    return l            

In [20]:
movies_df['cast'] = movies_df['cast'].apply(converter_cast)

In [21]:
def converte_crew(val):

    l = []
    for ele in ast.literal_eval(val):
        if ele['job'] == 'Director':
            l.append(ele['name'])
    return l 
    

In [22]:
movies_df['crew'] = movies_df['crew'].apply(converte_crew)

In [23]:
movies_df['overview'] = movies_df['overview'].apply(lambda x:x.split())

In [24]:
movies_df['genres'] = movies_df['genres'].apply(lambda x:[i.replace(" ","") for i in x]) 
movies_df['cast'] = movies_df['cast'].apply(lambda x:[i.replace(" ","") for i in x]) 
movies_df['crew'] = movies_df['crew'].apply(lambda x:[i.replace(" ","") for i in x]) 
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[i.replace(" ","") for i in x]) 

In [25]:
movies_df['tags'] = movies_df['overview'] + movies_df['keywords'] + movies_df['genres'] + movies_df['cast'] +movies_df['crew'] 

In [26]:
new_df = movies_df[['movie_id', 'title', 'tags']]

In [27]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1176\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [28]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1176\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [44]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [30]:
import nltk
from nltk.stem import PorterStemmer

In [41]:
ps = PorterStemmer()
def steming(obj):
    list1 = list(obj.split())
    l = []
    for i in list1:
        l.append(ps.stem(i))
    return " ".join(l)    
        

In [43]:
new_df['tags'] = new_df['tags'].apply(steming)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1176\1908341733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(steming)


In [49]:
import sklearn

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [55]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [57]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [62]:
cv.get_feature_names_out()

<function ndarray.tolist>

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
similarity = cosine_similarity(vectors)

In [67]:
similarity[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [92]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [95]:
recommend('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf
